In [91]:
# importa las librerias necesarias para manejar datos, realizar operaciones numericas y conectar con postgresql
import pandas as pd 
import numpy as np 
import psycopg2

In [92]:
# establece la conexion con la base de datos postgresql usando las credenciales y parametros especificados
conn = psycopg2.connect(
    dbname="reservas_proyecto",
    user = "postgres",
    password = "admin",
    host = "localhost",
    port = "5432")

In [93]:
# crea un cursor para ejecutar consultas en la base de datos
cur = conn.cursor() 

In [94]:
# ejecuta una consulta para obtener la version del servidor de la base de datos y muestra el resultado
cur.execute("SELECT version();") 
cur.fetchone()

('PostgreSQL 17.2 on x86_64-windows, compiled by msvc-19.42.34435, 64-bit',)

### Tabla ciudad

In [203]:
# carga el dataframe desde un archivo pickle y muestra una muestra aleatoria de 5 filas
df_reservas = pd.read_pickle("../data/reservas_hoteles_limpio.pickle")
df_reservas.sample(5)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
3108,ec369fc8-c903-4014-b2cf-6c0ac42a9ffa,3106,Berta,Garzón,berta.garzón@example.com,True,2025-02-21,2025-03-01,2025-03-02,135,85.00,ibis Madrid Alcobendas,NaN,Madrid
12325,e04e8397-611f-4057-91da-9f3cb8b99dfe,12269,Prudencio,Carrera,prudencio.carrera@example.com,False,2025-02-11,2025-03-01,2025-03-02,35,363.24,Hotel Torre Dorada,2.9,Madrid
7620,1bbf4862-d27e-46cb-94f2-615a3ebb8369,7592,David,Cadenas,david.cadenas@example.com,True,2025-02-21,2025-03-01,2025-03-02,103,102.00,ibis budget Madrid Vallecas,NaN,Madrid
14565,4c2cee9a-97e1-4004-a5e4-ed6b33a43394,14475,Primitiva,Luján,primitiva.luján@example.com,False,2025-02-09,2025-03-01,2025-03-02,15,240.63,Hotel Monte Verde,3.1,Madrid
11017,24c796c8-23fc-4e4c-87ff-c074c1d43b55,10969,Chita,Valero,chita.valero@example.com,False,2025-02-06,2025-03-01,2025-03-02,3,255.18,Hotel Mirador Real,3.0,Madrid


In [96]:
# crea un dataframe con los valores unicos de la columna "ciudad" y lo nombra "nombre_ciudad"
tabla_ciudad = pd.DataFrame(df_reservas["ciudad"].unique(), columns=["nombre_ciudad"])

tabla_ciudad

,nombre_ciudad
0,Madrid


In [97]:
# crea una lista de listas con los valores de la columna "nombre_ciudad" para su posterior insercion en la base de datos
data_to_insert = [[row["nombre_ciudad"]] for indice, row in tabla_ciudad.iterrows()]

In [98]:
# define la consulta SQL para insertar valores en la tabla "ciudad" usando un parametro (%s) para cada nombre de ciudad
insert_query = """
        INSERT INTO ciudad (nombre_ciudad)
        VALUES (%s)
"""

In [99]:
# ejecuta la consulta de insercion en la base de datos con los datos preparados y confirma los cambios
cur.executemany(insert_query, data_to_insert)
conn.commit()

### Tabla eventos 

In [171]:
# carga el archivo pickle de eventos en un dataframe y muestra una muestra aleatoria de 5 filas
df_eventos = pd.read_pickle("../data/datos_extraidos/tabla_eventos.pickle")
df_eventos.sample(5)

,nombre_evento,url_evento,codigo_postal,direccion,horario,organizacion,inicio_evento,fin_evento,ciudad
273,'Refugiados' por Myriam Laguía,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28019,CALLE ISAAC ALBENIZ 1,19:00,Biblioteca Pública Municipal Ana María Matute ...,2025-02-14,2025-03-13,Madrid
128,IX Muestra de Orfebrería Contemporánea,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28012,CALLE CONCEPCION JERONIMA 15,14:00,Imprenta Municipal - Artes del libro,2025-01-28,2025-03-23,Madrid
151,Ludoteca en Centro Cultural Fernando de los Ríos,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28047,CALLE CAMARENA 10,None,Centro Cultural Fernando de los Ríos (Latina),2024-10-04,2025-06-28,Madrid
212,Taller de cómic (Infantil),http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029,PLAZA DONOSO 5,17:30,Biblioteca Pública Municipal María Zambrano (T...,2025-01-08,2025-06-04,Madrid
144,Literatura con nombre de mujer: Club de lectura.,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28022,PLAZA EURIPIDES 1,10:00,Espacio de Igualdad Lucía Sánchez Saornil. San...,2025-01-09,2025-03-28,Madrid


In [172]:
# ejecuta una consulta SQL para obtener los nombres de las ciudades y sus respectivos IDs, y los guarda en un diccionario
cur.execute("SELECT nombre_ciudad, id_ciudad FROM ciudad")
ciudad_dict = dict(cur.fetchall()) 
ciudad_dict

{'Madrid': 1}

In [173]:
# crea la lista `data_to_insert` procesando el dataframe `df_eventos` para insertar los datos en la tabla de eventos, eliminando duplicados, convirtiendo el codigo postal y las fechas al formato adecuado, y asociando las ciudades con su `id`
data_to_insert = []
df_evento = df_eventos[["nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "inicio_evento", "fin_evento", "organizacion", "ciudad"]].drop_duplicates()
for _, row in df_eventos.iterrows(): 
    nombre_evento = row["nombre_evento"]
    url_evento = row["url_evento"]
    codigo_postal = row["codigo_postal"]
    codigo_postal = int(codigo_postal) if pd.notna(codigo_postal) else None
    direccion = row["direccion"]
    horario = row["horario"]
    fecha_inicio = pd.to_datetime(row["inicio_evento"])
    fecha_fin = pd.to_datetime(row["fin_evento"])
    organizacion = row["organizacion"]
    ciudad = row["ciudad"]
    id_ciudad = int(ciudad_dict.get(ciudad))
    data_to_insert.append([nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad]) 

# crea el dataframe `tabla_eventos` con la informacion procesada y lo muestra
tabla_eventos = pd.DataFrame(data_to_insert, columns=["nombre_evento", "url_evento", "codigo_postal", "direccion", "horario", "fecha_inicio", "fecha_fin", "organizacion", "id_ciudad"])
tabla_eventos.head()

,nombre_evento,url_evento,codigo_postal,direccion,horario,fecha_inicio,fecha_fin,organizacion,id_ciudad
0,25º aniversario de la revista La Fragua,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28005.0,CALLE SAN JUSTO 5,None,2025-02-24,2025-03-02,Biblioteca Pública Municipal Iván de Vargas (C...,1
1,60 Premio Reina Sofía de Pintura y Escultura,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28009.0,PASEO COLOMBIA 1,None,2025-02-27,2025-03-23,Centro Cultural Casa de Vacas (Retiro),1
2,A toda vela,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28045.0,PLAZA LEGAZPI 8,None,2024-10-01,2025-06-01,Matadero Madrid,1
3,Acompañamiento digital a personas mayores,http://www.madrid.es/sites/v/index.jsp?vgnextc...,NaN,None,None,2024-10-14,2025-06-30,None,1
4,Acompañar en la pérdida,http://www.madrid.es/sites/v/index.jsp?vgnextc...,28029.0,AVENIDA MONFORTE DE LEMOS 38,17:00,2025-01-14,2025-06-17,Biblioteca Pública Municipal José Saramago (Fu...,1


In [174]:
# prepara y ejecuta una consulta SQL para insertar multiples registros en la tabla `eventos` de la base de datos, luego confirma los cambios
insert_query = """
        INSERT INTO eventos (nombre_evento, url_evento, codigo_postal, direccion, horario, fecha_inicio, fecha_fin, organizacion, id_ciudad)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

### Tabla hoteles 

In [177]:
# filtra los hoteles que no son competencia y selecciona las columnas relevantes sin duplicados
df_reservas.head()
df_propios = df_reservas[df_reservas["competencia"] == False]
df_propios = df_propios[["nombre_hotel", "estrellas", "competencia", "ciudad"]].drop_duplicates()
df_propios

,nombre_hotel,estrellas,competencia,ciudad
2,Hotel Monte Verde,3.1,False,Madrid
3,Hotel Brisas del Mar,3.1,False,Madrid
4,Hotel Camino del Sol,3.1,False,Madrid
5,Hotel Puerta del Cielo,3.0,False,Madrid
6,Hotel Encanto Real,3.0,False,Madrid
8,Palacio del Sol,3.0,False,Madrid
10,Hotel Jardines del Rey,2.9,False,Madrid
13,Hotel Las Estrellas,2.9,False,Madrid
15,Gran Hotel Madrid,3.1,False,Madrid
17,Hotel Torre Dorada,2.9,False,Madrid


In [179]:
# carga los datos de competencia, selecciona las columnas relevantes y asigna valores para 'competencia' y 'ciudad'
df_competencia = pd.read_pickle("../data/datos_extraidos/nombre_estrellas_precio.pickle")
df_competencia = df_competencia[["nombre_hotel", "estrellas"]]
df_competencia["competencia"] = True
df_competencia["ciudad"] = "Madrid"
df_competencia

,nombre_hotel,estrellas,competencia,ciudad
0,ibis Styles Madrid Prado,4.7,True,Madrid
1,ibis budget Madrid Calle 30,4.4,True,Madrid
2,ibis Madrid Centro las Ventas,4.5,True,Madrid
3,ibis budget Madrid Centro las Ventas,4.3,True,Madrid
4,ibis budget Madrid Vallecas,4.3,True,Madrid
5,ibis Madrid Aeropuerto Barajas,4.4,True,Madrid
6,ibis Madrid Alcorcon Tresaguas,4.4,True,Madrid
7,ibis budget Madrid Aeropuerto,4.0,True,Madrid
8,ibis Madrid Alcobendas,4.4,True,Madrid
9,ibis budget Madrid Alcorcon Móstoles,4.5,True,Madrid


In [180]:
# combina los DataFrames de hoteles propios y competencia en uno solo, ignorando los indices originales
df_hoteles = pd.concat([df_propios, df_competencia], ignore_index=True)
df_hoteles

,nombre_hotel,estrellas,competencia,ciudad
0,Hotel Monte Verde,3.1,False,Madrid
1,Hotel Brisas del Mar,3.1,False,Madrid
2,Hotel Camino del Sol,3.1,False,Madrid
3,Hotel Puerta del Cielo,3.0,False,Madrid
4,Hotel Encanto Real,3.0,False,Madrid
5,Palacio del Sol,3.0,False,Madrid
6,Hotel Jardines del Rey,2.9,False,Madrid
7,Hotel Las Estrellas,2.9,False,Madrid
8,Gran Hotel Madrid,3.1,False,Madrid
9,Hotel Torre Dorada,2.9,False,Madrid


In [185]:
# extrae los datos relevantes del DataFrame de hoteles, los prepara para la insercion en la base de datos y los organiza en una nueva tabla con los valores adecuados
data_to_insert = []
df_hotel = df_hoteles[["nombre_hotel", "estrellas", "competencia", "ciudad"]]
for _, row in df_hoteles.iterrows(): 
    nombre_hotel = row["nombre_hotel"]
    estrellas = row["estrellas"]
    competencia = row["competencia"]
    ciudad = row["ciudad"]
    id_ciudad = int(ciudad_dict.get(ciudad))
    data_to_insert.append([nombre_hotel, estrellas, competencia, id_ciudad]) 

tabla_hoteles = pd.DataFrame(data_to_insert, columns=["nombre_hotel", "estrellas", "competencia", "id_ciudad"])
tabla_hoteles.head()

,nombre_hotel,estrellas,competencia,id_ciudad
0,Hotel Monte Verde,3.1,False,1
1,Hotel Brisas del Mar,3.1,False,1
2,Hotel Camino del Sol,3.1,False,1
3,Hotel Puerta del Cielo,3.0,False,1
4,Hotel Encanto Real,3.0,False,1


In [183]:
# inserta los datos de la tabla 'hoteles' en la base de datos utilizando un comando SQL y ejecuta la transaccion
insert_query = """
        INSERT INTO hoteles (nombre_hotel, estrellas, competencia, id_ciudad)
        VALUES (%s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

### Tabla clientes

In [212]:
# extrae y organiza los datos unicos de los clientes de df_reservas para crear un nuevo DataFrame tabla_clientes
data_to_insert = []
df_clientes = df_reservas[["id_cliente", "nombre", "apellido", "mail"]].drop_duplicates()
for _, row in df_clientes.iterrows(): 
    id_cliente = row["id_cliente"]
    nombre = row["nombre"]
    apellido = row["apellido"]
    mail = row["mail"]
    data_to_insert.append([id_cliente, nombre, apellido, mail]) 

tabla_clientes = pd.DataFrame(data_to_insert, columns=["id_cliente", "nombre", "apellido", "mail"])
tabla_clientes

,id_cliente,nombre,apellido,mail
0,1,Feliciana,Cantón,feliciana.cantón@example.com
1,2,Leonardo,Hierro,leonardo.hierro@example.com
2,3,Maite,Calatayud,maite.calatayud@example.com
3,4,Tecla,Bonet,tecla.bonet@example.com
4,5,Amílcar,Andrés,amílcar.andrés@example.com
...,...,...,...,...
14900,14901,Lorena,Román,lorena.román@example.com
14901,14902,Toño,Narváez,toño.narváez@example.com
14902,14903,Nadia,Verdugo,nadia.verdugo@example.com
14903,14904,Custodia,Marco,custodia.marco@example.com


In [201]:
# inserta los datos de los clientes en la tabla "clientes" de la base de datos usando la consulta SQL definida
insert_query = """
        INSERT INTO clientes (id_cliente, nombre, apellido, mail)
        VALUES (%s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

### Tabla reservas

In [205]:
# ejecuta una consulta SQL para obtener los nombres de los hoteles y sus respectivos id, y los almacena en un diccionario
cur.execute("SELECT nombre_hotel, id_hotel FROM hoteles")
hotel_dict = dict(cur.fetchall()) 
hotel_dict

{'Hotel Monte Verde': 1,
 'Hotel Brisas del Mar': 2,
 'Hotel Camino del Sol': 3,
 'Hotel Puerta del Cielo': 4,
 'Hotel Encanto Real': 5,
 'Palacio del Sol': 6,
 'Hotel Jardines del Rey': 7,
 'Hotel Las Estrellas': 8,
 'Gran Hotel Madrid': 9,
 'Hotel Torre Dorada': 10,
 'Hotel Palacio Imperial': 11,
 'Hotel Luz de Madrid': 12,
 'Hotel Los Almendros': 13,
 'Hotel Sol y Luna': 14,
 'Hotel Mirador Real': 15,
 'Hotel Rincón Sereno': 16,
 'Hotel Vista Alegre': 17,
 'Hotel Costa Azul': 18,
 'Hotel Maravilla Real': 19,
 'ibis Styles Madrid Prado': 20,
 'ibis budget Madrid Calle 30': 21,
 'ibis Madrid Centro las Ventas': 22,
 'ibis budget Madrid Centro las Ventas': 23,
 'ibis budget Madrid Vallecas': 24,
 'ibis Madrid Aeropuerto Barajas': 25,
 'ibis Madrid Alcorcon Tresaguas': 26,
 'ibis budget Madrid Aeropuerto': 27,
 'ibis Madrid Alcobendas': 28,
 'ibis budget Madrid Alcorcon Móstoles': 29}

In [206]:
# ejecuta una consulta SQL para obtener los correos electronicos y los id de los clientes, y los almacena en un diccionario
cur.execute("SELECT mail, id_cliente FROM clientes")
cliente_dict = dict(cur.fetchall()) 
cliente_dict

{'feliciana.cantón@example.com': '1',
 'leonardo.hierro@example.com': '2',
 'maite.calatayud@example.com': '3',
 'tecla.bonet@example.com': '4',
 'amílcar.andrés@example.com': '5',
 'joan.vazquez@example.com': '6',
 'chelo.flor@example.com': '7',
 'judith.rivas@example.com': '8',
 'dolores.lobo@example.com': '9',
 'adolfo.cuervo@example.com': '10',
 'noé.lobo@example.com': '11',
 'maría carmen.navarro@example.com': '12',
 'ricardo.caballero@example.com': '13',
 'omar.lamas@example.com': '14',
 'luna.segovia@example.com': '15',
 'ágata.pinedo@example.com': '16',
 'lázaro.pardo@example.com': '17',
 'julie.baños@example.com': '18',
 'javier.mancebo@example.com': '19',
 'rita.guerrero@example.com': '20',
 'ricardo.lobo@example.com': '21',
 'perlita.patiño@example.com': '22',
 'pedro.gallego@example.com': '23',
 'judith.bellido@example.com': '24',
 'francisco javier.hierro@example.com': '25',
 'reynaldo.lerma@example.com': '26',
 'felicidad.gimenez@example.com': '27',
 'pía.barba@example.co

In [215]:
# crea un DataFrame con las reservas unicas, luego itera para insertar datos y asignar los ids correspondientes de clientes y hoteles
data_to_insert = []
df_tabla_reservas = df_reservas[["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "mail", "nombre_hotel"]]
for _, row in df_tabla_reservas.iterrows(): 
    id_reserva = row["id_reserva"]
    fecha_reserva = row["fecha_reserva"]
    inicio_estancia = row["inicio_estancia"]
    final_estancia = row["final_estancia"]
    precio_noche = row["precio_noche"]
    mail = row["mail"]
    id_cliente = cliente_dict.get(mail)
    nombre_hotel = row["nombre_hotel"]
    id_hotel = hotel_dict.get(nombre_hotel)
    data_to_insert.append([id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel]) 

tabla_reservas = pd.DataFrame(data_to_insert, columns=["id_reserva", "fecha_reserva", "inicio_estancia", "final_estancia", "precio_noche", "id_cliente", "id_hotel"])
tabla_reservas.head()

,id_reserva,fecha_reserva,inicio_estancia,final_estancia,precio_noche,id_cliente,id_hotel
0,5256cc90-139b-43d2-8ec5-412495d751cf,2025-02-21,2025-03-01,2025-03-02,161.00,1,20
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,2025-02-21,2025-03-01,2025-03-02,110.00,2,21
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,2025-02-09,2025-03-01,2025-03-02,119.96,3,1
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,2025-02-08,2025-03-01,2025-03-02,211.53,4,2
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,2025-02-02,2025-03-01,2025-03-02,378.37,5,3


In [210]:
# inserta multiples filas de reservas en la tabla de la base de datos utilizando la consulta SQL con los datos preparados
insert_query = """
        INSERT INTO reservas (id_reserva, fecha_reserva, inicio_estancia, final_estancia, precio_noche, id_cliente, id_hotel)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

cur.executemany(insert_query, data_to_insert)
conn.commit()

In [217]:
# cierra la conexion a la base de datos y el cursor
cur.close()
conn.close()